# Hands on Object Relational Mapping in PyCram

This tutorial will walk through the serialization of a minimal plan in pycram.
First we will import sqlalchemy, create an in memory database and connect a session to it.

In [1]:
import sqlalchemy
import sqlalchemy.orm

engine = sqlalchemy.create_engine("sqlite+pysqlite:///:memory:", echo=False)
session = sqlalchemy.orm.Session(bind=engine)
session

Next we create the database schema using the sqlalchemy functionality. For that we need to import the base class of pycram.orm.

In [2]:
import pycram.orm.base
import pycram.orm.task
import pycram.orm.object_designator
import pycram.orm.motion_designator
import pycram.orm.action_designator
pycram.orm.base.Base.metadata.create_all(engine)
session.commit()

Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']


shutdown request: [/pycram] Reason: new node registered with same name


Next we will write a simple plan where the robot parks his arms and then moves somewhere. We will construct a TaskTree around it such that we can serialize it later. As usual, we first create a world and then define the plan. After that we get and print the task tree.

In [ ]:
from pycram.designators.action_designator import *
from pycram.designators.location_designator import *
from pycram.process_module import simulated_robot
from pycram.resolver.plans import Arms
from pycram.task import with_tree
import pycram.task
from pycram.bullet_world import BulletWorld, Object
from pycram.robot_descriptions.robot_description_handler import InitializedRobotDescription as robot_description
from pycram.designators.object_designator import *
import anytree

world = BulletWorld()
robot = Object(robot_description.i.name, "robot", robot_description.i.name + ".urdf")
kitchen = Object("kitchen", "environment", "kitchen.urdf")
milk = Object("milk", "milk", "milk.stl", position=[1.3, 1, 0.9])
cereal = Object("cereal", "cereal", "breakfast_cereal.stl", position=[1.3, 0.7, 0.95])
milk_desig = ObjectDesignator(ObjectDesignatorDescription(name="milk", type="milk"))
cereal_desig = ObjectDesignator(ObjectDesignatorDescription(name="cereal", type="cereal"))

@with_tree
def plan():
    with simulated_robot:
        ActionDesignator(ParkArmsAction(Arms.BOTH)).perform()
        ActionDesignator(MoveTorsoAction(0.3)).perform()
        location = LocationDesignator(CostmapLocation(target=milk, reachable_for=robot))
        pose = location.reference()
        ActionDesignator(
            NavigateAction(target_position=pose["position"], target_orientation=pose["orientation"])).perform()
        ActionDesignator(ParkArmsAction(Arms.BOTH)).perform()

        picked_up_arm = pose["arms"][0]
        ActionDesignator(
            PickUpAction(object_designator=milk_desig, arm=pose["arms"][0], grasp="front")).perform()

        place_island = LocationDesignator(
            SemanticCostmapLocation("kitchen_island_surface", kitchen, milk_desig.prop_value("object")))
        pose_island = place_island.reference()

        place_location = LocationDesignator(
            CostmapLocation(target=list(pose_island.values()), reachable_for=robot,
                            reachable_arm=picked_up_arm))
        pose = place_location.reference()

        ActionDesignator(
            NavigateAction(target_position=pose["position"], target_orientation=pose["orientation"])).perform()

        ActionDesignator(PlaceAction(object_designator=milk_desig, target_location=list(pose_island.values()),
                                     arm=picked_up_arm)).perform()

        ActionDesignator(ParkArmsAction(Arms.BOTH)).perform()

plan()
world.exit()

task_tree = pycram.task.task_tree
print(anytree.RenderTree(task_tree))



Next we serialize the task tree by just recursively inserting from its root.

In [4]:
task_tree.root.insert(session)

pycram.orm.task.TaskTreeNode(1, 1, 2023-04-17 10:37:41.648917, None, RUNNING, None)

Lastly we can look at various table to see how the structures got logged.
For example, we can get all the navigate actions that occurred.

In [5]:
navigations = session.query(pycram.orm.action_designator.NavigateAction).all()
print(*navigations, sep="\n")


pycram.orm.action_designator.NavigateAction(3, Navigate, 1, 1)
pycram.orm.action_designator.NavigateAction(6, Navigate, 3, 3)


Since inheritance is correctly mapped in the ORM package we can also get all actions that were executed with the correct classes in just one line.

In [6]:
actions = session.query(pycram.orm.action_designator.Action).all()
print(*actions, sep="\n")

pycram.orm.action_designator.ParkArmsAction(1, ParkArms, BOTH)
pycram.orm.action_designator.MoveTorsoAction(2, MoveTorso, 0.3)
pycram.orm.action_designator.NavigateAction(3, Navigate, 1, 1)
pycram.orm.action_designator.ParkArmsAction(4, ParkArms, BOTH)
pycram.orm.action_designator.PickUpAction(5, PickUp, left, front, 1, 0.9)
pycram.orm.action_designator.NavigateAction(6, Navigate, 3, 3)
pycram.orm.action_designator.PlaceAction(7, Place, left, 5, 5, 2)
pycram.orm.action_designator.ParkArmsAction(8, ParkArms, BOTH)


Of course all relational algebra operators, such as filtering and joining also work in pycram.orm queries. For example if we want all actions that have an object designator assigned in them, we can execute:

In [7]:
object_actions = session.query(pycram.orm.action_designator.PickUpAction,
                               pycram.orm.object_designator.ObjectDesignator).join(pycram.orm.object_designator.ObjectDesignator).all() + \
                 session.query(pycram.orm.action_designator.PlaceAction,
                               pycram.orm.object_designator.ObjectDesignator).join(pycram.orm.object_designator.ObjectDesignator).all()
print(*object_actions, sep="\n")

(pycram.orm.action_designator.PickUpAction(5, PickUp, left, front, 1, 0.9), pycram.orm.object_designator.ObjectDesignator(1, Object, milk, milk, 2, 2))
(pycram.orm.action_designator.PlaceAction(7, Place, left, 5, 5, 2), pycram.orm.object_designator.ObjectDesignator(2, Object, milk, milk, 4, 4))


If we want to filter for all successful tasks we can just add the filter operator.

In [8]:
successful_tasks = session.query(pycram.orm.task.TaskTreeNode).filter(pycram.orm.task.TaskTreeNode.status == "SUCCEEDED")
print(*successful_tasks, sep="\n")

pycram.orm.task.TaskTreeNode(2, 2, 2023-04-17 10:37:51.368893, 2023-04-17 10:37:56.650215, SUCCEEDED, 1)
pycram.orm.task.TaskTreeNode(3, 3, 2023-04-17 10:37:51.368941, 2023-04-17 10:37:51.871805, SUCCEEDED, 2)
pycram.orm.task.TaskTreeNode(4, 4, 2023-04-17 10:37:51.871897, 2023-04-17 10:37:52.372631, SUCCEEDED, 2)
pycram.orm.task.TaskTreeNode(5, 5, 2023-04-17 10:37:53.049817, 2023-04-17 10:37:53.551284, SUCCEEDED, 2)
pycram.orm.task.TaskTreeNode(6, 6, 2023-04-17 10:37:53.551344, 2023-04-17 10:37:54.052426, SUCCEEDED, 2)
pycram.orm.task.TaskTreeNode(7, 7, 2023-04-17 10:37:54.052499, 2023-04-17 10:37:54.557173, SUCCEEDED, 2)
pycram.orm.task.TaskTreeNode(8, 8, 2023-04-17 10:37:55.131750, 2023-04-17 10:37:55.633911, SUCCEEDED, 2)
pycram.orm.task.TaskTreeNode(9, 9, 2023-04-17 10:37:55.633974, 2023-04-17 10:37:56.148843, SUCCEEDED, 2)
pycram.orm.task.TaskTreeNode(10, 10, 2023-04-17 10:37:56.148917, 2023-04-17 10:37:56.650206, SUCCEEDED, 2)


As expected all but the root node succeeded, since the root node is still running.

Writing an extension to the ORM package is also done with ease. We need to create a class that can be inserted and its ORM equivalent, write a to_sql() and insert() method and then insert it somewhere.

In [9]:
import pycram.designators.action_designator


# define ORM class from pattern in every pycram.orm class
class ORMSaying(pycram.orm.action_designator.Action):
    __tablename__ = "Saying"
    id = sqlalchemy.Column(sqlalchemy.types.Integer, sqlalchemy.ForeignKey("Action.id"), primary_key=True)
    text = sqlalchemy.Column(sqlalchemy.types.String(255))

    __mapper_args__ = {
        "polymorphic_identity": __tablename__,
        "polymorphic_on": "dtype",
    }

    def __init__(self, text: str):
        super().__init__()
        self.text = text


# define brand new action designator
class Saying(pycram.designators.action_designator.ActionDesignatorDescription):

    def __init__(self, text):
        super().__init__()
        self.text = text

    def to_sql(self):
        return ORMSaying(self.text)

    def insert(self, session):
        action = self.to_sql()
        session.add(action)
        session.commit()
        return action


So we now got our new ActionDesignator called Saying and its ORM version. Since this class got created after all other classes got inserted into the database (in the beginning of the notebook) we have to insert it manually.

In [10]:
ORMSaying.metadata.create_all(bind=engine)

Now we can create and insert a Saying action.

In [11]:
# create a saying action and insert it
foo = Saying("Patchie, Patchie; Where is my Patchie?")
orm_foo = foo.insert(session)
orm_foo

__main__.ORMSaying(9, Saying, Patchie, Patchie; Where is my Patchie?)

It is notable that committing the object to the session fills its primary key. Hence, there is no worries about assigning unique IDs manually.
Finally, we can double-check that our object exists in the database.

In [12]:
session.query(ORMSaying).all()

[__main__.ORMSaying(9, Saying, Patchie, Patchie; Where is my Patchie?)]